In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchinfo import summary
from torcheval.metrics import MulticlassAccuracy

import numpy as np
import wandb

In [2]:
num_classes = 37
def get_data_set(batch_size):
    #
    # CenterCrop is one possibility, but you can also try to resize the image
    #
    transform = torchvision.transforms.Compose(
            [torchvision.transforms.ToTensor(),
             torchvision.transforms.CenterCrop(256)])
    data_train = torchvision.datasets.OxfordIIITPet(root='data/OxfordIIITPet', download=True, transform=transform)
    data_test = torchvision.datasets.OxfordIIITPet(root='data/OxfordIIITPet', split='test', download=True,
                                                   transform=transform)
    len_train = (int)(0.8 * len(data_train))
    len_val = len(data_train) - len_train
    data_train_subset, data_val_subset = torch.utils.data.random_split(
            data_train, [len_train, len_val])

    data_train_loader = torch.utils.data.DataLoader(dataset=data_train_subset, shuffle=True, batch_size=batch_size)
    data_val_loader = torch.utils.data.DataLoader(dataset=data_val_subset, shuffle=True, batch_size=batch_size)
    data_test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

    print(f'Length of Datasets: train-{len_train}, val- {len_val}, test-{len(data_train)}')

    return data_train_loader, data_val_loader, data_test_loader

In [3]:
class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        # to complete
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),
            nn.Linear(512 * 8 * 8, 256),
            nn.ReLU(),

            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        # to complete
        return self.layers(x)

In [1]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    run = wandb.init(project="DeepCNN_PetClassification", config={'epochs': num_epochs, 
                                                       'batch_size': train_loader.batch_size}
                                                       )
    metrics = MulticlassAccuracy(num_classes=37)
    total_step = len(train_loader)
    
    for epoch in range(num_epochs):
        model.to(device)
        model.train()
        metrics.reset()

        for step, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            metrics.update(predicted, labels)
            train_acc = metrics.compute()

            train_metrics = \
                {'train/train_loss:': loss.item(),
                 'train/train_acc'        : train_acc,
                 'train/epoch'            : epoch+1}

            if (step+1) % 10 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], '
                       f'Step [{step+1}/{total_step}], '
                       f'Loss: {loss.item(): .4f}, '
                       f'Accuracy: {train_acc: .2f}')
                #log loss and accuracy to wandb
                wandb.log(train_metrics)

        model.eval()
        with torch.no_grad():
            metrics.reset()
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                metrics.update(predicted, labels)
            val_acc = metrics.compute()
            val_metrics = {'val/val_loss' : val_loss_mean,
                           'val/val_acc'       : val_acc}

            print(f'Val Accuracy: {val_acc: .2f}')
            #log Validation accuracy to wandb
            wandb.log(val_metrics)

    return model

In [10]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        # test if it worked
        x = torch.ones(1, device=device)
        print('Using CUDA device')

    elif torch.backends.mps.is_available():
        device = torch.device('mps')
        x = torch.ones(1, device=device)
        print('Using MPS device')
    else:
        print('Using CPU')
        device = torch.device('cpu')
    return device

In [11]:


def main():
    batch_size = 64
    train_loader, val_loader, test_loader = get_data_set(batch_size)
    # to complete
    device = get_device()
    cnn = DeepCNN()
    n_parameters = sum(p.numel() for p in cnn.parameters())
    print(f'Number of Parameters: {n_parameters}')

    n_epochs = 20
    lr = 0.01
    weight_decay = 0.001
    optimizer = optim.Adam(cnn.parameters(), lr=lr, weight_decay=weight_decay)
    loss_func = nn.CrossEntropyLoss()

    final_model, results = train(cnn, train_loader, val_loader, loss_func, optimizer, n_epochs, device)
    wandb.finish()

In [12]:
#
# Exercise 02 for advanced deep learning course
#
#
# Construct a deep CNN model for Pet Classification
#
#
# This version does not use wandb, but tensorboard or wandb are recommended
#
if __name__ == '__main__':
    main()


Length of Datasets: train-2944, val- 736, test-3680
Using CPU
Number of Parameters: 9968933


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: itsshilpi26 (g27). Use `wandb login --relogin` to force relogin


Epoch [1/20], Step [10/46], Loss:  67.1935, Accuracy:  0.04
Epoch [1/20], Step [20/46], Loss:  5.0284, Accuracy:  0.03
Epoch [1/20], Step [30/46], Loss:  3.6077, Accuracy:  0.02
Epoch [1/20], Step [40/46], Loss:  3.5962, Accuracy:  0.02
Val Accuracy:  0.02
Epoch [2/20], Step [10/46], Loss:  3.6333, Accuracy:  0.04
Epoch [2/20], Step [20/46], Loss:  3.7119, Accuracy:  0.03
Epoch [2/20], Step [30/46], Loss:  3.6081, Accuracy:  0.03
Epoch [2/20], Step [40/46], Loss:  3.6010, Accuracy:  0.03
Val Accuracy:  0.02
Epoch [3/20], Step [10/46], Loss:  3.5878, Accuracy:  0.03
Epoch [3/20], Step [20/46], Loss:  3.6080, Accuracy:  0.04
Epoch [3/20], Step [30/46], Loss:  3.6574, Accuracy:  0.03
Epoch [3/20], Step [40/46], Loss:  3.6104, Accuracy:  0.03
Val Accuracy:  0.02
Epoch [4/20], Step [10/46], Loss:  3.6079, Accuracy:  0.04
Epoch [4/20], Step [20/46], Loss:  3.7976, Accuracy:  0.03
Epoch [4/20], Step [30/46], Loss:  3.7118, Accuracy:  0.03
Epoch [4/20], Step [40/46], Loss:  4.4953, Accuracy:  

TypeError: cannot unpack non-iterable DeepCNN object